## 作業

請使用 ImageDataGenerator 來進行 Cifar-10 資料集的訓練，並觀察不同的圖像增強方法是否會顯著影響訓練結果

In [1]:
from tensorflow import keras
batch_size = 32
num_classes = 10
epochs = 5

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=[32,32,3]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
#model.summary()

In [3]:
model.save_weights("tmp.h5")

## Training without data augmentation

In [4]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
def cifar_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels
            
cifar_gen = cifar_generator(len(x_train), batch_size)

In [6]:
model.load_weights("tmp.h5")
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0805 21:29:42.402786  2008 deprecation.py:323] From c:\users\qwerz\miniconda3\envs\ml100\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
1562/1562 [==============================] - 26s 17ms/step - loss: 1.7428 - accuracy: 0.3563 - val_loss: 1.3774 - val_accuracy: 0.4960
Epoch 2/5
1562/1562 [==============================] - 24s 15ms/step - loss: 1.2763 - accuracy: 0.5441 - val_loss: 1.0317 - val_accuracy: 0.6332
Epoch 3/5
1562/1562 [==============================] - 24s 15ms/step - loss: 1.0438 - accuracy: 0.6323 - val_loss: 0.9940 - val_accuracy: 0.6510
Epoch 4/5
1562/1562 [==============================] - 24s 15ms/step - loss: 0.8872 - accuracy: 0.6873 - val_loss: 0.8857 - val_accuracy: 0.6937
Epoch 5/5
1562/1562 [==============================] - 25s 16ms/step - loss: 0.7877 - accuracy: 0.7229 - val_loss: 0.7855 - val_accuracy: 0.7321
Test loss: 0.7851886442661286
Test accuracy: 0.7321


## Training with data augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_test = x_test.astype('float32')
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
def cifar_aug_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            images = next(data_generator.flow(images, shuffle=False))
            images = images.astype('float32')
            images /= 255
            yield images, labels
             
cifar_gen = cifar_aug_generator(len(x_train), batch_size)

In [10]:
model.load_weights("tmp.h5")
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
1562/1562 [==============================] - 33s 21ms/step - loss: 1.9304 - accuracy: 0.2801 - val_loss: 1.6178 - val_accuracy: 0.3966
Epoch 2/5
1562/1562 [==============================] - 31s 20ms/step - loss: 1.6120 - accuracy: 0.4058 - val_loss: 1.4166 - val_accuracy: 0.4983
Epoch 3/5
1562/1562 [==============================] - 30s 19ms/step - loss: 1.4557 - accuracy: 0.4702 - val_loss: 1.2350 - val_accuracy: 0.5695
Epoch 4/5
1562/1562 [==============================] - 30s 19ms/step - loss: 1.3496 - accuracy: 0.5145 - val_loss: 1.1275 - val_accuracy: 0.5965
Epoch 5/5
1562/1562 [==============================] - 30s 19ms/step - loss: 1.2548 - accuracy: 0.5490 - val_loss: 1.0396 - val_accuracy: 0.6359
Test loss: 1.039435496711731
Test accuracy: 0.6359
